# 疫情地图
- [疫情地图](https://www.dnablockchain.cn/news.htm)，[百度迁徙图](https://qianxi.baidu.com/?from=shoubai#city=420100)
- 以下代码参考：[用Python制作实时疫情图](https://www.lizenghai.com/archives/46326.html)，采用[Pyecharts](https://pyecharts.org/#/zh-cn/geography_charts)绘制

In [2]:
import requests, re, json
import pyecharts as pe
import pprint as pp 
result = requests.get(
        'https://interface.sina.cn/news/wap/fymap2020_data.d.json?1580097300739&&callback=sinajp_1580097300873005379567841634181')
json_str = re.search("\(+([^)]*)\)+", result.text).group(1)
#print json_str
html = f"{json_str}"
table = json.loads(f"{html}")
#table = json.loads(json_str)
#table

In [14]:

prov_list = []
for i in table['data']['list']:
    prov = i['name']
    prov_list.append(prov)
    #print(prov)
    #print(prov, data['gntotal'][prov], data['sustotal'], data['deathtotal'], data['curetotal'])
print(prov_list)
import pandas as pd

df = pd.DataFrame()
df['省份'] = prov_list
for k in table['data']['list']['city']:
    print(k)
    df[val_dict[k][0]] = [i[1] for i in table['data']['list'][k]]
df.headd()

['北京', '湖北', '广东', '浙江', '河南', '湖南', '重庆', '安徽', '四川', '山东', '吉林', '福建', '江西', '江苏', '上海', '广西', '海南', '陕西', '河北', '黑龙江', '辽宁', '云南', '天津', '山西', '甘肃', '内蒙古', '台湾', '澳门', '香港', '贵州', '西藏', '青海', '新疆', '宁夏']


TypeError: list indices must be integers or slices, not str

In [16]:
data['gntotal']

[('北京', '387'),
 ('湖北', '59989'),
 ('广东', '1328'),
 ('浙江', '1172'),
 ('河南', '1257'),
 ('湖南', '1007'),
 ('重庆', '553'),
 ('安徽', '982'),
 ('四川', '508'),
 ('山东', '543'),
 ('吉林', '89'),
 ('福建', '292'),
 ('江西', '933'),
 ('江苏', '629'),
 ('上海', '333'),
 ('广西', '242'),
 ('海南', '163'),
 ('陕西', '240'),
 ('河北', '302'),
 ('黑龙江', '464'),
 ('辽宁', '121'),
 ('云南', '172'),
 ('天津', '128'),
 ('山西', '130'),
 ('甘肃', '91'),
 ('内蒙古', '73'),
 ('台湾', '22'),
 ('澳门', '10'),
 ('香港', '62'),
 ('贵州', '146'),
 ('西藏', '1'),
 ('青海', '18'),
 ('新疆', '76'),
 ('宁夏', '70')]

In [13]:
# 全国信息
val_dict = {'gntotal': ['确诊', 'value'], 
            'sustotal': ['疑似', 'susNum'], 
            'deathtotal': ['死亡', 'deathNum'], 
            'curetotal': ['治愈', 'cureNum']}
data = {'gntotal':[], 'sustotal':[], 'deathtotal':[], 'curetotal':[]}
# 各城市信息
city_info = {}
#city_key = ['name', 'conNum', 'susNum', 'deathNum', 'cureNum'] #, 'mapName']
city_dict = {'conNum':'确诊', 'susNum':'疑似', 'deathNum':'死亡', 'cureNum':'治愈'}
for province in table['data']['list']:
    #pp.pprint(province)
    for k, v in val_dict.items(): 
        data[k].append((province['name'], province[v[1]]))
    city_info[province['name']] = dict([(c, []) for c in city_dict.keys()])
    city_info[province['name']]['all'] = dict([(val_dict[i][1], province[val_dict[i][1]]) for i in val_dict])
    city_info[province['name']]['all']['conNum'] = city_info[province['name']]['all']['value']
    for city in province['city']:
        # [{'name': '杭州', 'conNum': '141','susNum': '0', 'cureNum': '14','deathNum': '0','mapName': '杭州市'},...]
        #pp.pprint(city)
        for c_num in city_dict:
            city_info[province['name']][c_num].append((city['mapName'], city[c_num]))

# 地图绘制
page = pe.charts.Page() # 瀑布方式
map_p = pe.charts.Map() # 全国地图
opts = pe.options # 配置信息
# 数据处理：数据范围（自适应）
# （1）全国地图
num_list = [int(i[1]) for i in data['gntotal']]
print(num_list)
num_range = [min(num_list), max(num_list)]
num_range[1] = sorted(num_list)[-2] * 1.2 # 以第二名的数字的1.2倍为上界，便于可视化
map_p.set_global_opts(title_opts=opts.TitleOpts(
                        title="2020年新型冠状病毒实时疫情图（全国）\n\t确诊：{}，疑似：{}，治愈：{}，死亡：{}".format(
                            table['data']['gntotal'], 
                            table['data']['sustotal'], 
                            table['data']['curetotal'],
                            table['data']['deathtotal']),
                        subtitle='鹤啸九天-{}'.format(table['data']['times'])),
                      #legend_opts=opts.TooltipOpts(),
                      toolbox_opts=opts.ToolboxOpts(),
                      visualmap_opts=opts.VisualMapOpts(min_=num_range[0], max_=num_range[1])) #, is_piecewise=True))
for k, v in val_dict.items():
    print(k, data[k])
    map_p.add(v[0], data[k], maptype="china") # maptype="湖北"
#page.add(map_p, grid_opts=opts.GridOpts(pos_bottom="60%"))
page.add(map_p) # 添加大图里
# （2）城市地图（覆盖全国所有省市自治区）
p_list = ['湖北', '北京', '广东' , '浙江']
map_c = {}
print('city_info', city_info.keys())
for c in city_info:
    # 获取数据范围
    num_list = [int(i[1]) for i in city_info[c]['conNum']]
    if not num_list:
        print('{}省数据为空！'.format(c))
        continue
    num_range = [min(num_list), max(num_list)]
    if len(num_list) < 2:
        num_range[1] = num_list[0] * 1.2
    else:
        sort_list = sorted(num_list)
        if sort_list[-1] / sort_list[-2] > 5:
            num_range[1] = sort_list[-2] * 1.2 # 第一名差距过大时，以第二名的数字的1.2倍为上界，便于可视化
    if num_range[1] < 20: # 最大值较小时，将最小值设置为0，提升区分度
        num_range[0] = 0
    map_c = pe.charts.Map()
    map_c.set_global_opts(title_opts=opts.TitleOpts(
                            title="2020年新型冠状病毒实时疫情图（{}）\n\t确诊：{}，疑似：{}，死亡：{}，治愈：{}".format(c,
                                city_info[c]['all']['conNum'], 
                                city_info[c]['all']['susNum'], 
                                city_info[c]['all']['deathNum'],
                                city_info[c]['all']['cureNum']),
                            subtitle='鹤啸九天-{}'.format(table['data']['times'])), 
                          toolbox_opts=opts.ToolboxOpts(),
                          visualmap_opts=opts.VisualMapOpts(min_=num_range[0], max_=num_range[1]))
    for k, v in city_dict.items():
        map_c.add(v, city_info[c][k], maptype='{}'.format(c)) # maptype="湖北"
    page.add(map_c)
# 输出
page.render("wqw_ncov.html")  # 生成html文件
page.render_notebook() # notebook中直接显示

[387, 59989, 1328, 1172, 1257, 1007, 553, 982, 508, 543, 89, 292, 933, 629, 333, 242, 163, 240, 302, 464, 121, 172, 128, 130, 91, 73, 22, 10, 62, 146, 1, 18, 76, 70]
gntotal [('北京', '387'), ('湖北', '59989'), ('广东', '1328'), ('浙江', '1172'), ('河南', '1257'), ('湖南', '1007'), ('重庆', '553'), ('安徽', '982'), ('四川', '508'), ('山东', '543'), ('吉林', '89'), ('福建', '292'), ('江西', '933'), ('江苏', '629'), ('上海', '333'), ('广西', '242'), ('海南', '163'), ('陕西', '240'), ('河北', '302'), ('黑龙江', '464'), ('辽宁', '121'), ('云南', '172'), ('天津', '128'), ('山西', '130'), ('甘肃', '91'), ('内蒙古', '73'), ('台湾', '22'), ('澳门', '10'), ('香港', '62'), ('贵州', '146'), ('西藏', '1'), ('青海', '18'), ('新疆', '76'), ('宁夏', '70')]
sustotal [('北京', '0'), ('湖北', '0'), ('广东', '0'), ('浙江', '0'), ('河南', '0'), ('湖南', '0'), ('重庆', '0'), ('安徽', '0'), ('四川', '0'), ('山东', '0'), ('吉林', '34'), ('福建', '0'), ('江西', '0'), ('江苏', '0'), ('上海', '97'), ('广西', '0'), ('海南', '0'), ('陕西', '79'), ('河北', '6'), ('黑龙江', '101'), ('辽宁', '0'), ('云南', '0'), ('天津', '0'), ('山

## Debug区域
- 以下代码调试使用

In [140]:
num_list = [int(i[1]) for i in data['gntotal']]
print(sorted(num_list))
print(num_list)
num_range = (min(num_list), max(num_list))
num_range
#table
#table['data']['list'][3]
#city_info

[1, 10, 16, 18, 24, 36, 40, 46, 59, 62, 71, 78, 90, 94, 108, 133, 157, 168, 173, 215, 227, 257, 274, 321, 347, 373, 400, 591, 600, 711, 851, 954, 970, 19665]
[274, 19665, 970, 954, 851, 711, 400, 591, 321, 347, 59, 215, 600, 373, 257, 168, 108, 173, 157, 227, 94, 133, 78, 90, 62, 46, 16, 10, 24, 71, 1, 18, 36, 40]


(1, 19665)

In [83]:
a=['x','y']
b=dict([(k, []) for k in a])
b

{'x': [], 'y': []}

In [141]:
from pyecharts.faker import Faker
from pyecharts import options as opts
from pyecharts.charts import Bar, Grid, Line,Scatter


def grid_vertical() -> Grid:
    bar = (
        Bar()
        .add_xaxis(Faker.choose())
        .add_yaxis("商家A", Faker.values())
        .add_yaxis("商家B", Faker.values())
        .set_global_opts(title_opts=opts.TitleOpts(title="Grid-Bar"))
    )
    line = (
        Line()
        .add_xaxis(Faker.choose())
        .add_yaxis("商家A", Faker.values())
        .add_yaxis("商家B", Faker.values())
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Grid-Line", pos_top="48%"),
            legend_opts=opts.LegendOpts(pos_top="48%"),
        )
    )

    grid = (
        Grid()
        .add(bar, grid_opts=opts.GridOpts(pos_bottom="60%"))
        .add(line, grid_opts=opts.GridOpts(pos_top="60%"))
    )
    return grid

#grid_vertical().render_notebook()


In [167]:
import numpy as np
# initialize the population vector from the origin-destination flow matrix
N_k = np.abs(np.diagonal(OD) + OD.sum(axis=0) - OD.sum(axis=1))
locs_len = len(N_k)                 # number of locations
SIR = np.zeros(shape=(locs_len, 3)) # make a numpy array with 3 columns for keeping track of the S, I, R groups
SIR[:,0] = N_k                      # initialize the S group with the respective populations
first_infections = np.where(SIR[:, 0]<=thresh, SIR[:, 0]//20, 0)   # for demo purposes, randomly introduce infections
SIR[:, 0] = SIR[:, 0] - first_infections
SIR[:, 1] = SIR[:, 1] + first_infections# move infections to the I group

# row normalize the SIR matrix for keeping track of group proportions
row_sums = SIR.sum(axis=1)
SIR_n = SIR / row_sums[:, np.newaxis]

# initialize parameters
beta = 1.6
gamma = 0.04
public_trans = 0.5  # alpha
R0 = beta/gamma
beta_vec = np.random.gamma(1.6, 2, locs_len)
gamma_vec = np.full(locs_len, gamma)
public_trans_vec = np.full(locs_len, public_trans)

# make copy of the SIR matrices 
SIR_sim = SIR.copy()
SIR_nsim = SIR_n.copy()

# run model
print(SIR_sim.sum(axis=0).sum() == N_k.sum())
from tqdm import tqdm_notebook
infected_pop_norm = []
susceptible_pop_norm = []
recovered_pop_norm = []
for time_step in tqdm_notebook(range(100)):
    infected_mat = np.array([SIR_nsim[:,1],]*locs_len).transpose()
    OD_infected = np.round(OD*infected_mat)
    inflow_infected = OD_infected.sum(axis=0)
    inflow_infected = np.round(inflow_infected*public_trans_vec)
    print('total infected inflow: ', inflow_infected.sum())
    new_infect = beta_vec*SIR_sim[:, 0]*inflow_infected/(N_k + OD.sum(axis=0))
    new_recovered = gamma_vec*SIR_sim[:, 1]
    new_infect = np.where(new_infect>SIR_sim[:, 0], SIR_sim[:, 0], new_infect)
    SIR_sim[:, 0] = SIR_sim[:, 0] - new_infect
    SIR_sim[:, 1] = SIR_sim[:, 1] + new_infect - new_recovered
    SIR_sim[:, 2] = SIR_sim[:, 2] + new_recovered
    SIR_sim = np.where(SIR_sim<0,0,SIR_sim)
    # recompute the normalized SIR matrix
    row_sums = SIR_sim.sum(axis=1)
    SIR_nsim = SIR_sim / row_sums[:, np.newaxis]
    S = SIR_sim[:,0].sum()/N_k.sum()
    I = SIR_sim[:,1].sum()/N_k.sum()
    R = SIR_sim[:,2].sum()/N_k.sum()
    print(S, I, R, (S+I+R)*N_k.sum(), N_k.sum())
    print('\n')
    infected_pop_norm.append(I)
    susceptible_pop_norm.append(S)
    recovered_pop_norm.append(R)

NameError: name 'OD' is not defined